In [ ]:
from langchain.text_splitter import MarkdownTextSplitter

In [1]:
import chromadb
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.node_parser import SentenceSplitter,MarkdownNodeParser ,SentenceWindowNodeParser
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.llms.glm import GLM
from llama_index.embeddings.hk_embedding import BGEEmbedding

In [4]:
llms = GLM()
embedding = BGEEmbedding()

In [11]:
embedding.get_text_embedding("""你好""")

[-0.037132903933525085,
 0.006196123082190752,
 -0.06525908410549164,
 -0.025046681985259056,
 -0.016926361247897148,
 -0.04781854897737503,
 -0.002269513439387083,
 -0.025503840297460556,
 -0.014202593825757504,
 0.011753924190998077,
 0.01296042837202549,
 0.02710241824388504,
 0.006390976253896952,
 -0.0015952145913615823,
 0.012333323247730732,
 -0.020321592688560486,
 -0.005955481436103582,
 -0.029828136786818504,
 -0.00809423066675663,
 -0.040239498019218445,
 -0.02898889034986496,
 -0.012031348422169685,
 0.004369657952338457,
 0.008861811831593513,
 0.019466185942292213,
 0.03204254433512688,
 -0.05138825625181198,
 -0.0034972289577126503,
 0.03346497192978859,
 -0.0031682197004556656,
 0.043934646993875504,
 0.011451958678662777,
 0.006940491497516632,
 -0.04224018752574921,
 -0.016590077430009842,
 -0.0460439957678318,
 0.00690578855574131,
 -0.02429981529712677,
 -0.03685544803738594,
 0.035123277455568314,
 0.0344572588801384,
 0.01804545894265175,
 0.035826295614242554,
 -

In [5]:
laws = SimpleDirectoryReader(input_dir=""".\\Law-Book""",exclude=['md'],recursive=True).load_data()

In [6]:
spliter = MarkdownNodeParser(separator="""\n\n""")

In [7]:
nodes = spliter.get_nodes_from_documents(documents=laws,show_progress=True)

Parsing nodes:   0%|          | 0/2850 [00:00<?, ?it/s]

In [8]:
db = chromadb.PersistentClient(path = """.\\LawDb""")


In [9]:
db.create_collection("laws")

Collection(name=laws)

In [12]:
chroma_collection = db.get_or_create_collection("laws")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
store_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(
    nodes=nodes,
    embed_model=embedding,
    storage_context=store_context,
    show_progress=True,
)

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Exception: Embedding request failed: Internal Server Error

In [20]:
b = [i.text for i in nodes]

In [24]:
len(b)

2850

In [23]:
embedding.get_text_embedding(b[1])

[0.0074680945836007595,
 0.040285464376211166,
 -0.03913285210728645,
 -0.00021477683912962675,
 -0.028948422521352768,
 -0.04327959567308426,
 0.0021596583537757397,
 0.011773318983614445,
 0.003359591355547309,
 0.05879557132720947,
 -0.033571287989616394,
 0.01880861259996891,
 0.0062227630987763405,
 -0.004200948402285576,
 0.0023420273791998625,
 0.002667077351361513,
 0.061192791908979416,
 -0.005308317951858044,
 -8.2405298599042e-05,
 0.045075803995132446,
 -0.017552774399518967,
 -0.019849073141813278,
 0.0030344268307089806,
 -0.008158725686371326,
 -0.001971582183614373,
 0.0054686288349330425,
 0.0006913096876814961,
 0.031348489224910736,
 0.0070550511591136456,
 -0.027150992304086685,
 -0.03274752199649811,
 0.039863500744104385,
 0.025968249887228012,
 0.012900462374091148,
 -0.03883056342601776,
 -0.017529230564832687,
 -0.010267741978168488,
 -0.027864590287208557,
 -0.010317537933588028,
 0.011054274626076221,
 0.03423222899436951,
 0.021552694961428642,
 -0.007823892

In [10]:
memory = ChatMemoryBuffer.from_defaults()

In [21]:
chat_engine = index.as_chat_engine(
    llm=llms,
    memory=memory,
    chat_mode="condense_plus_context",
    context_prompt=(
        """
        你是一个聊天机器人，能够进行正常互动，还可以回答法律问题,请保持回答简洁。
        以下是相关背景资料：{context_str} 
        指令：利用之前的聊天记录或上述背景资料进行互动，帮助用户。
        """
    ),
    verbose=False
)

In [12]:
chat_engine.reset()

In [22]:
response = chat_engine.stream_chat("""你好""")
for token in response.response_gen:
    print(token, end="")

你好！有什么法律问题需要帮助吗？

In [32]:
def chat(inputer:str,reset=False):
    if reset:
        chat_engine.reset()
    response = chat_engine.stream_chat(inputer)
    for token in response.response_gen:
        # print(, end="")
        display.display(display.Markdown(token))

In [29]:


async def chat(inputer: str, reset=False):
    if reset:
        chat_engine.reset()
    response = await chat_engine.astream_chat(inputer)
    for token in response.response_gen:
        print(token, end="")

# 调用异步函数

In [34]:



def chat(inputer: str, reset: bool = False):
    if reset:
        chat_engine.reset()

    # 初始化一个字符串来收集输出
    collected_output = ""

    # 获取流式输出
    response = chat_engine.stream_chat(inputer)

    # 收集输出
    for token in response.response_gen:
        collected_output += token

    # 使用display.Markdown来显示收集到的Markdown格式内容
    display.display(display.Markdown(collected_output))


# 假设chat_engine已经定义，并且有stream_chat和reset方法
# chat_engine = ...

In [27]:
import IPython.display as display

# 创建一个Markdown字符串
markdown_str = """
| 标题   | 描述                                           |
| ------ | ---------------------------------------------- |
| **加粗** | 这里的文字是**加粗**的。                       |
| *斜体*  | 这里的文字是*斜体*的。                         |
| `代码`  | 这里的文字是`代码`样式。                       |
| [链接](http://example.com) | 这里的文字是一个[链接](http://example.com)。 |
"""

# 显示Markdown格式的文本



| 标题   | 描述                                           |
| ------ | ---------------------------------------------- |
| **加粗** | 这里的文字是**加粗**的。                       |
| *斜体*  | 这里的文字是*斜体*的。                         |
| `代码`  | 这里的文字是`代码`样式。                       |
| [链接](http://example.com) | 这里的文字是一个[链接](http://example.com)。 |


In [36]:
chat("""关于故意和过失的司法解释？""",True)

在中国法律中，故意和过失是两种不同的主观过错状态，用于判断行为人是否应当承担侵权责任。

1. **故意**：指行为人明知自己的行为会造成损害结果，并且希望或者放任这种结果发生。故意侵权行为的责任通常较重。

2. **过失**：指行为人应当预见自己的行为可能造成损害结果，但由于疏忽大意而没有预见，或者虽然预见到了但轻信能够避免。过失侵权行为的责任相对较轻。

在侵权责任法中，行为人因过错（包括故意和过失）侵害他人民事权益造成损害的，应当承担侵权责任。如果法律规定推定行为人有过错，而行为人不能证明自己没有过错的，也应当承担侵权责任。

In [7]:
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)

AttributeError: 'StorageContext' object has no attribute 'as_chat_engine'